## Monitoring data quality in third-party models from the AWS Marketplace

**Overview:**
The process of implementing a third-party model generally starts with searching for a model that meets your business needs, then thoroughly evaluating and testing the model (using your own ground truth dataset), and finally deploying the model to production. If the statistical nature of the data that your model receives while in production drifts away from the nature of the baseline data it was evaluated on, the model might begin to lose accuracy in its predictions. 

Amazon SageMaker's [Data quality monitoring](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-quality.html) monitors machine learning (ML) models in production and notifies you when data quality issues arise.  Amazon SageMaker Model Monitor uses rules to detect data drift and alerts you when it happens. These alerts will help you understand whether you would need to re-evalute the ML model to see if it is still providing you the correct outputs. If it is not, you would need to request the third party seller to retrain and provide a new version of the AWS Marketplace model. 

In this notebook, you will learn how to perform Data Quality monitoring on a pre-trained third-party model from the AWS Marketplace.

**Contents:**
- Pre-requisites
- Step 1. Initial setup
    - 1.1 [Import packages and modules](#section_1_1)
    - 1.2 [Set global variables](#section_1_2)
    - 1.3 [Uploading sample datasets to your S3 bucket](#section_1_3)
- Step 2. Create and deploy the model endpoint with data capture
    - 2.1 [Create the model](#section_2_1)
    - 2.2 [Create the endpoint configuration with DataCapture](#section_2_2)
    - 2.3 [Create the model endpoint](#section_2_3)
    - 2.4 [Periodically check if the model's endpoint has changed from 'Creating' to 'InService'](#section_2_4)
- Step 3. Create a baselining job to suggest a set of baseline constraints
    - 3.1 [Create baselining job](#section_3_1)
- Step 4. Setup a monitoring schedule to monitor the data captured for the model's endpoint
    - 4.1 [Create a monitoring schedule](#section_4_1)
- Step 5. Invoking the inference endpoint with anomalous data
    - 5.1 [Create a wrapper function for the Predictor](#section_5_1)
    - 5.2 [Generate some data quality constraint violations](#section_5_2)
    - 5.3 [Check the model monitoring processing job for completion](#section_5_3)
- Step 6. Invoking the inference endpoint with anomalous data
    - 6.1 [Visualize the model monitor results](#section_6_1)
    - 6.2 [Import the SageMaker-Model-Monitor-Visualize.ipynb notebook](#section_6_2)
    - 6.3 [Run the SageMaker-Model-Monitor-Visualize.ipynb notebook](#section_6_3)    

**Pre-requisites**

This notebook requires a subscription to the [Propensity-Planning to Buy a House](https://aws.amazon.com/marketplace/pp/prodview-vzofptk4lnxii) model, a pre-trained machine learning model package from AWS Marketplace.

1. Open the [Propensity-Planning to Buy a House](https://aws.amazon.com/marketplace/pp/prodview-vzofptk4lnxii) model in your browser. 

2. To subscribe to the model package, follow these steps: 
  1. Review the information available on the product details page including **Support Terms** .
  1. Click on **"Continue to Subscribe"**. You will now see the **"Subscribe to this software"** page. 
  1. Review **End User License Agreement** and **Pricing Terms**.
  1. **"Accept Offer"** button needs to be clicked if your organization agrees with EULA, pricing information and support terms.
  1. Once you click on **Continue to configuration** button and then choose a region, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3.  Copy the Model Package ARN and replace its contents in following cell. 
  
 
Note: 
Products with **Free Trials**, do not incur hourly software charges during free trial period, but AWS infrastructure charges still apply. Free Trials will automatically convert to a paid hourly subscription upon expiration. We have included steps below to cancel subscription at the end of this exercise. 


In [ ]:
#MODEL_PACKAGE_ARN=''
MODEL_PACKAGE_ARN = 'arn:aws:sagemaker:us-east-1:865070037744:model-package/planning-to-buy-house-basic-28fcb3ca751705854a7171b255d8ef43'  # Update as needed

<a id=section_1_1></a>
#### 1.1 Import packages and modules

In [ ]:
import boto3
import pandas as pd
import requests
import time
import io

from time import gmtime, strftime

from sagemaker import get_execution_role
from sagemaker import session
from sagemaker import ModelPackage

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

from sagemaker.model_monitor import CronExpressionGenerator, DataCaptureConfig, DefaultModelMonitor, MonitoringExecution
from sagemaker.model_monitor.dataset_format import DatasetFormat

<a id=section_1_2></a>
#### 1.2 Set global variables

In [ ]:
# Get IAM role and SakeMaker session
role = get_execution_role()
session = session.Session()

# Set service clients
s3_client = boto3.client('s3')
sm_client = boto3.client('sagemaker')
smr_client = boto3.client('sagemaker-runtime')

# S3
BUCKET = session.default_bucket() # Update as needed
PREFIX = 'third-party-model-seller-name' # Update as needed

S3_DATA_CAPTURE_URI = 's3://{}/{}/datacapture'.format(BUCKET, PREFIX)
S3_BASELINE_DATASET_URI = 's3://{}/{}/train/{}'.format(BUCKET, PREFIX, 'baseline.csv')
S3_BASELINE_ANALYSIS_RESULTS_URI = 's3://{}/{}/baselining'.format(BUCKET, PREFIX)
S3_DATA_QUALITY_RPT_URI = 's3://{}/{}/reports'.format(BUCKET, PREFIX)

# Model
MODEL_ENDPOINT = 'third-party-model-endpoint'
MODEL_BASELINE_JOB = 'third-party-model-baseline-job'
MODEL_MONITOR_SCHEDULE_NAME = 'third-party-model-data-quality-schedule'
MODEL_MONITOR_INSTANCE_TYPE = 'ml.m4.xlarge'
MODEL_INFERENCE_INSTANCE_TYPE = 'ml.m4.xlarge'
MODEL_INSTANCE_COUNT = 1
MODEL_BASELINE_JOB_NAME = '{}-{}'.format(MODEL_BASELINE_JOB, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

<a id=section_1_3></a>
#### 1.3 Uploading sample datasets to your S3 bucket

The [baseline.csv](https://raw.githubusercontent.com/william-screen/model-monitor/main/third-party-models/prosper/data/baseline.csv) and [data_quality_drift.csv](https://raw.githubusercontent.com/william-screen/model-monitor/main/third-party-models/prosper/data/data_quality_drift.csv) files are curated datasets I created by modifying the seller's [sample notebook](https://github.com/goprosper/prosper-sagemaker-basic-geo/blob/master/using_prosper_model_package_basic_geo.ipynb) to return high propensity and low propensity values (respectively) for generating a baseline_drift_check constraint violation.

Additionally, the data_quality_drift.csv file contains some rows with anomolous data that are used to demonstrate data_type_check and completeness_check constraint violations.

The baseline.csv file contains 29 columns and 301 rows with the first column as the target value (prediction)

The data_quality_drift.csv file contains 28 columns and 200 rows and is designed to illustrate a degredation in data quality from the baseline dataset.

For reference, here's the folder structure used for this demo if you're planning to upload the datasets and run the notebook:

<pre>
+---data
|       baseline.csv
|       data_quality_drift.csv
+---notebooks
|       monitoring_data_quality_of_models.ipynb
</pre>

In [ ]:
with open('../data/baseline.csv', "rb") as f:
    s3_client.upload_fileobj(f, BUCKET, '{}/train/{}'.format(PREFIX, 'baseline.csv'))

In [ ]:
with open('../data/data_quality_drift.csv', "rb") as f:
    s3_client.upload_fileobj(f, BUCKET, '{}/train/{}'.format( PREFIX, 'data_quality_drift.csv'))

<a id=section_2_1></a>
#### 2.1 Create the model

Creates a model in Amazon SageMaker from a model package.

In [ ]:
#Create a deployable model for damage inspection model package.
model = ModelPackage(role=role, model_package_arn=MODEL_PACKAGE_ARN, sagemaker_session=session)

<a id=section_2_2></a>
#### 2.2 Deploy model while enabling DataCapture

Creates an endpoint configuration that Amazon SageMaker hosting services uses to deploy models.

In [21]:
%%time

# Set data capture configuration settings
data_capture_config = DataCaptureConfig(enable_capture=True,
                                        sampling_percentage=100,
                                        destination_s3_uri=S3_DATA_CAPTURE_URI,
                                        csv_content_types=["text/csv"])

# Deploy the model
model.deploy(initial_instance_count=MODEL_INSTANCE_COUNT,
             instance_type=MODEL_INFERENCE_INSTANCE_TYPE,
             endpoint_name=MODEL_ENDPOINT,
             data_capture_config=data_capture_config)

--!CPU times: user 298 ms, sys: 26 ms, total: 324 ms
Wall time: 9min 32s


<a id=section_3_1></a>
#### 3.1 Create a baselining job

The baseline calculations of statistics and constraints are needed as a standard against which data drift and other data quality issues can be detected. This job generates baseline statistics and suggests baseline constraints for the dataset and writes them to the output_s3_uri location that you specify.

In [22]:
%%time

# Initializes a Monitor instance
default_model_monitor = DefaultModelMonitor(role=role,
                                            instance_count=MODEL_INSTANCE_COUNT,
                                            instance_type=MODEL_MONITOR_INSTANCE_TYPE)

# Suggest baselines for use with Amazon SageMaker Model Monitoring Schedules
job = default_model_monitor.suggest_baseline(job_name = MODEL_BASELINE_JOB_NAME,
                                             baseline_dataset=S3_BASELINE_DATASET_URI,
                                             dataset_format=DatasetFormat.csv(header=True),
                                             output_s3_uri=S3_BASELINE_ANALYSIS_RESULTS_URI,
                                             wait=True,
                                             logs=False)


Job Name:  third-party-model-baseline-job-v4-2021-01-19-14-32-54
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-717870172810/third-party-model-seller-name-v4/train/baseline.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-717870172810/third-party-model-seller-name-v4/baselining', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................................................................!CPU times: user 448 ms, sys: 28.9 ms, total: 477 ms
Wall time: 6min 1s


<a id=section_4_1></a>
#### 4.1 Create a monitoring schedule

In this step, you create a schedule that starts Amazon SageMaker Processing Jobs every hour to monitor the data captured for an Amazon SageMaker Endoint.

**Note:** Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start anywhere between the first ~20 minutes after the hour boundary (i.e. 00:00 – 00:20). This is expected and done for load balancing on the backend.

In [23]:
# Creates a schedule that regularly starts a processing jobs to monitor the data captured for the model's endoint
default_model_monitor.create_monitoring_schedule(
    monitor_schedule_name = MODEL_MONITOR_SCHEDULE_NAME,
    endpoint_input = MODEL_ENDPOINT,
    output_s3_uri = S3_DATA_QUALITY_RPT_URI,
    statistics = default_model_monitor.baseline_statistics(),
    constraints = default_model_monitor.suggested_constraints(),
    schedule_cron_expression = CronExpressionGenerator.hourly())

# Allow time for processing
time.sleep(30)

# Print the current status
monitor_schedule_details = default_model_monitor.describe_schedule()['MonitoringScheduleStatus']
print('>> The current status of monitoring schedule "{0}" is {1}'.format(MODEL_MONITOR_SCHEDULE_NAME, monitor_schedule_details))

>> The current status of monitoring schedule "third-party-model-data-quality-schedule-v4" is Scheduled


<a id=section_5_1></a>
#### 5.1 Create a wrapper function for the Predictor

The Predictor object is used to invoke the model's real-time inference endpoint to make predictions based on sample data. This function adds some defensive coding, a time delay, and decodes the response.

In [24]:
# Create predictor endpoint
predictor = Predictor(endpoint_name=MODEL_ENDPOINT, sagemaker_session=None, serializer=CSVSerializer())

In [25]:
def predict(sample, delay=0.5):
    """Return the inference from the model's endpoint.
    Args:
        sample (str): Sample data from client
        delay (float): The number of seconds execution is suspended
    Returns:
        (string): inference data from the model's endpoint
    """    
    if(len(sample) > 0):
        time.sleep(delay)
        return predictor.predict(data=sample).decode('utf-8')

<a id=section_5_2></a>
#### 5.2 Generate some data quality constraint violations

For this demo, we'll generate a couple of [data quality constraint violations](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-violations.html) (data_type_check, completeness_check, and baseline_drift_check) for Amazon SageMaker Model Monitor to detect.

Here's the structure of the data_quality_drift.csv file:

- Rows 1-10:  Samples with negative floating point values (instead of positive integers) in the 3rd and 4th columns
- Rows 11-20: Samples with missing values in the 2nd column
- Rows 20-200: Samples that yield a low propensity prediction that deviates from the baseline dataset

We'll use array slicing for accessing the anomolous sample data.

In [26]:
# Convert to Dataframe
df = pd.read_csv('../data/data_quality_drift.csv', header=None, na_filter=False)

# List values by column to maintain the column dtype
samples = [df[x].values.tolist() for x in df.columns]

# Use unpacking operator * to unzip the data
samples = list(list(x) for x in zip(*samples))

Here we'll generate a data_type_check constraint violation

In [27]:
# Invoke real-time inference endpoint 
for index, sample in enumerate(samples[0:10]):

    # Get inference response
    response = predict(sample)

    # Display the model's prediction probability
    print('Sample {0} >> Input: {1}: >> Prediction: {2}'.format(index, sample, response))

Sample 0 >> Input: [1, '3', 22, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]: >> Prediction: 0.6004737615585327
Sample 1 >> Input: [1, '3', 21, -0.25, -42.0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.5937598347663879
Sample 2 >> Input: [1, '4', 24, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]: >> Prediction: 0.6058259606361389
Sample 3 >> Input: [1, '4', 24, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.6058259606361389
Sample 4 >> Input: [1, '3', 21, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]: >> Prediction: 0.5937598347663879
Sample 5 >> Input: [1, '3', 20, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]: >> Prediction: 0.5937598347663879
Sample 6 >> Input: [1, '3', 20, -0.25, -42.0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

Now, let's generate a completeness_check constraint violation

In [28]:
# Invoke real-time inference endpoint 
for index, sample in enumerate(samples[10:20]):

    # Get inference response
    response = predict(sample)
    
    # Display the model's prediction probability
    print('Sample {0} >> Input: {1}: >> Prediction: {2}'.format(index, sample, response))

Sample 0 >> Input: [1, '', 22, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]: >> Prediction: 0.5373708009719849
Sample 1 >> Input: [1, '', 21, 0.0, 0.0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.5304257273674011
Sample 2 >> Input: [1, '', 24, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]: >> Prediction: 0.5373708009719849
Sample 3 >> Input: [1, '', 24, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.5373708009719849
Sample 4 >> Input: [1, '', 21, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]: >> Prediction: 0.5304257273674011
Sample 5 >> Input: [1, '', 20, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]: >> Prediction: 0.5304257273674011
Sample 6 >> Input: [1, '', 20, 0.0, 0.0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.53

Finally, we'll generate a baseline_drift_check constraint violation

In [29]:
# Invoke real-time inference endpoint 
for index, sample in enumerate(samples[20:]):

    # Get inference response
    response = predict(sample)

    # Display the model's prediction probability
    print('Sample {0} >> Input: {1}: >> Prediction: {2}'.format(index, sample, response))

Sample 0 >> Input: [1, '6', 5, 0.0, 0.0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.45879191160202026
Sample 1 >> Input: [1, '3', 1, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]: >> Prediction: 0.5094481706619263
Sample 2 >> Input: [1, '4', 1, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]: >> Prediction: 0.49388447403907776
Sample 3 >> Input: [1, '5', 6, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]: >> Prediction: 0.4816320240497589
Sample 4 >> Input: [0, '2', 9, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]: >> Prediction: 0.5360277891159058
Sample 5 >> Input: [1, '6', 9, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]: >> Prediction: 0.45879191160202026
Sample 6 >> Input: [0, '5', 14, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]: >> Prediction: 

<a id=section_5_3></a>
#### 5.3 Check the model monitoring processing job for completion

Please wait for the first data drift monitoring job to complete. Execution may start anywhere between the first ~20 minutes after the hour boundary (i.e. 00:00 – 00:20). This is expected and done for load balancing on the backend.

In [30]:
# Get the first monitoring execution details
monitoring_executions = sm_client.list_monitoring_executions(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)

# Check if monitoring job has started
while not monitoring_executions.get('MonitoringExecutionSummaries'):
    
    # Progress update
    print("Waiting for the data drift model monitoring processing job to start...")
    
    # Pause for 60 seconds
    time.sleep(60)
    
    # Get the first monitoring execution details
    monitoring_executions = sm_client.list_monitoring_executions(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)

print("The data drift model monitoring processing job has started.")

Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
The data drift model monitoring processing job has started.


Periodically check the job's status. Upon the job's successful completion, print the ProcessingJobArn for use in section 6.3.

In [31]:
# Check for the first monitoring execution detail (update as needed for subsequent monitoring executions)
if(len(monitoring_executions['MonitoringExecutionSummaries']) == 1):
    
    # Periodically check the processing job status 
    while True:
        
        # Get the first monitoring execution details
        monitoring_executions = sm_client.list_monitoring_executions(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)    
        monitoring_execution_details = monitoring_executions['MonitoringExecutionSummaries'][0]  
        
        # Get the processing job status
        processing_job_status = monitoring_execution_details.get('MonitoringExecutionStatus')
        
        # Periodically check job status
        if(processing_job_status != 'InProgress' and processing_job_status != 'Pending'):
            
            # Progress update
            print("\nProcessing Job Status: {}".format(processing_job_status))
            print("Processing Job Arn: {}".format(monitoring_execution_details.get('ProcessingJobArn')))            
            break

        # Progress update
        print("Processing Job Status: {}".format(processing_job_status))   
            
        # Pause for 60 seconds
        time.sleep(60)

Processing Job Status: Pending
Processing Job Status: Pending
Processing Job Status: InProgress
Processing Job Status: InProgress
Processing Job Status: InProgress
Processing Job Status: InProgress
Processing Job Status: InProgress
Processing Job Status: InProgress

Processing Job Status: CompletedWithViolations
Processing Job Arn: arn:aws:sagemaker:us-east-1:717870172810:processing-job/model-monitoring-202101191500-ac2000d03f76a75bf08671ab


<a id=section_6_1></a>
#### 6.1 Visualize the model monitor results

After the data drift monitoring job has run, select Endpoints from the Sagemaker Studio Components and registries console, then double-click the endpoint name to view the Model Monitoring tab. 

The Monitoring status of ‘Issue found’ indicates that the monitor successfully detected one or more data quality constraint violations created by the data drift datasets. 

Double-click the monitoring job name to view the Model Job Details tab.

<img src="../images/s4-1-sm-studio-visualize-results.png"
    alt="Amazon Sagemaker Data Drift Monitoring"
    style="float: left; margin-right: 10px;" />



<img src="../images/s4-2-monitoring-job-report-details.png"
    alt="Amazon Sagemaker Model Job Report"
    style="float: left; margin-right: 10px;" />

<a id=section_6_2></a>
#### 6.2 Import the SageMaker-Model-Monitor-Visualize.ipynb notebook

To graphically visualize the distribution and the distribution statistics for all features, SakeMaker includes a pre-built notebook for viewing feature statistics.

1. In the Model Job Details tab under Feature Statistics, select 'View Amazon SageMaker notebook'
2. Select Import Notebook (in the upper-right section of the tab)
3. Select Kernel: Python 3 (Data Science), then choose the Select button. (It may take a few minutes for the Kernel to start)

<a id=section_6_3></a>
#### 6.3 Run the SageMaker-Model-Monitor-Visualize.ipynb notebook

1. After importing the SageMaker-Model-Monitor-Visualize.ipynb notebook, update the code cell that contains the variable processing_job_arn with the value from the Processing Job ARN from above.
2. Run all cells in the notebook to review the execution and baseline details from the model monitoring processing job.

<img src="../images/s4-3-mm_visualize_inf_vs_baseline_stats-v2.png"
    alt="SageMaker Model Monitor Visualize notebook - Numerical Features"
    style="float: center; margin-right: 10px;" />

Tabular view of some basic statistical details such as mean, sum, standard deviation, etc... 

<img src="../images/s4-3-mm_visualize_inf_vs_baseline_stats_chart-v2.png"
    alt="SageMaker Model Monitor Visualize notebook - Inference feature statistics plots"
    style="float: center; margin-right: 10px;" />

Bar charts illustrating the deviation in statistical data quality between the baseline data and the data drift dataset.
The x-axis shows the range of collected inference data versus the baseline data. The y-axis shows the frequency of values.

### Conclusion

This notebook demonstrated how to subscribe to a pre-trained third-party model from the AWS Marketplace and configure a Data Quality monitoring schedule using Amazon SageMaker Model Monitor.

### Cleanup Resources

In [ ]:
# Stop the monitoring schedule
sm_client.stop_monitoring_schedule(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)
time.sleep(30) # allow time for processing

In [ ]:
# Delete the model monitoring schedule
sm_client.delete_monitoring_schedule(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)
time.sleep(30) # allow time for processing

In [ ]:
# Delete the model endpoint
sm_client.delete_endpoint(EndpointName=MODEL_ENDPOINT)
time.sleep(60) # allow time for processing

In [ ]:
# Delete the model endpoint config
sm_client.delete_endpoint_config(EndpointConfigName=MODEL_ENDPOINT)
time.sleep(30) # allow time for processing

If you would like to unsubscribe to the model, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you would need to cancel subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

